In [59]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
# import cvxopt

In [60]:
# class SVMRegression:
#     def __init__(self, C=1, kernel='linear', degree=3, gamma='scale'):
#         self.C = C
#         self.kernel = kernel
#         self.degree = degree
#         self.gamma = gamma
#         self.support_vectors_ = None
#         self.support_vectors_y_ = None

#     def fit(self, X, y):
#         n_samples, n_features = X.shape

#         # Compute the kernel matrix
#         K = np.zeros((n_samples, n_samples))
#         for i in range(n_samples):
#             for j in range(n_samples):
#                 K[i,j] = self.kernel_function(X[i], X[j])

#         # # Define the quadrbatic programming problem
#         # P = np.outer(y, y) * K
#         # q = -np.ones(n_samples)
#         # G = np.vstack((np.eye(n_samples)*-1, np.eye(n_samples)))
#         # h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C))

#         # # Solve the quadratic programming problem using an optimizer
        
#         # P, q, G, h = matrix(P), matrix(q), matrix(G), matrix(h)
#         # solvers.options['show_progress'] = False
#         # solution = solvers.qp(P, q, G, h)

#         # # Get the Lagrange multipliers
#         # lagrange_multipliers = np.array(solution['x']).flatten()


#      # Initialize Lagrange multipliers and error cache
#         lagrange_multipliers = np.zeros(n_samples)
#         error_cache = np.zeros(n_samples)

#         # Train the SVM regression model using SMO algorithm
#         num_changed = 0
#         examine_all = True
#         iteration = 0
#         while num_changed > 0 or examine_all:
#             num_changed = 0
#             if examine_all:
#                 for i in range(n_samples):
#                     num_changed += self.examine_example(i, X, y, K, lagrange_multipliers, error_cache)
#             else:
#                 non_bound_indices = np.where((lagrange_multipliers > 0) & (lagrange_multipliers < self.C))[0]
#                 for i in non_bound_indices:
#                     num_changed += self.examine_example(i, X, y, K, lagrange_multipliers, error_cache)
#             if examine_all:
#                 examine_all = False
#             elif num_changed == 0:
#                 examine_all = True
#             iteration += 1
#             if iteration >= self.max_iter:
#                 break



#         # Select the support vectors
#         threshold = 1e-5
#         support_vectors = lagrange_multipliers > threshold
#         self.support_vectors_ = X[support_vectors]
#         self.support_vectors_y_ = y[support_vectors]
#         self.lagrange_multipliers_ = lagrange_multipliers[support_vectors]

#         # Compute the bias term
#         self.intercept_ = np.mean(self.support_vectors_y_ - np.sum(self.lagrange_multipliers_ * self.support_vectors_y_ * K[support_vectors, :], axis=0))

#     def kernel_function(self, x1, x2):
#         if self.kernel == 'linear':
#             return np.dot(x1, x2)
#         elif self.kernel == 'poly':
#             return (np.dot(x1, x2) + 1) ** self.degree
#         elif self.kernel == 'rbf':
#             return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)
        



#     def examine_example(self, i, X, y, K, lagrange_multipliers, error_cache):
#         y_i = y[i]
#         alpha_i = lagrange_multipliers[i]
#         E_i = self.predict(X[i]) - y_i
#         r_i = E_i * y_i

#         if ((r_i < -self.tol and alpha_i < self.C) or (r_i > self.tol and alpha_i > 0)):
#             non_bound_indices = np.where((lagrange_multipliers > 0) & (lagrange_multipliers < self.C))[0]
#             if len(non_bound_indices) > 1:
#                 j = self.select_second_example(i, E_i, error_cache)
#                 if self.take_step

#     def predict(self, X):
#         n_samples, n_features = X.shape

#         # Compute the kernel matrix
#         K = np.zeros((n_samples, len(self.support_vectors_)))
#         for i in range(n_samples):
#             for j in range(len(self.support_vectors_)):
#                 K[i,j] = self.kernel_function(X[i], self.support_vectors_[j])

#         # Make predictions
#         y_pred = np.sum(self.lagrange_multipliers_ * self.support_vectors_y_ * K, axis=1) + self.intercept_

#         return y_pred


In [61]:
import numpy as np

class SVMRegression:
    def __init__(self, C=1, learning_rate=0.01, num_iterations=1000):
        self.C = C
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None
        
    def fit(self, X, y):
        # Convert labels to -1/+1
        y = np.where(y == 0, -1, 1)
        
        # Initialize weights and bias to zero
        self.weights = np.zeros(X.shape[1])
        self.bias = 0
        
        # Gradient descent optimization
        for iteration in range(self.num_iterations):
            # Compute gradients
            grad_weights = np.zeros(X.shape[1])
            grad_bias = 0
            for i in range(X.shape[0]):
                if y[i] * (np.dot(self.weights, X[i]) + self.bias) < 1:
                    grad_weights += self.C * y[i] * X[i]
                    grad_bias += self.C * y[i]
                else:
                    grad_weights += 0
                    grad_bias += 0
                    
            # Update weights and bias
            self.weights -= self.learning_rate * grad_weights
            self.bias -= self.learning_rate * grad_bias
            
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


In [62]:
modelEncoder = LabelEncoder()
transmissionEncoder = LabelEncoder()
fuelTypeEncoder = LabelEncoder()
scaler = MinMaxScaler()

def dataset(brand):
    file = pd.read_csv(brand, quotechar='"', skipinitialspace=True)

    modelEncoder.fit(file["model"])
    file["model"] = modelEncoder.transform(file["model"])
    
    transmissionEncoder.fit(file["transmission"])
    file["transmission"] = transmissionEncoder.transform(file["transmission"])
    
    fuelTypeEncoder.fit(file["fuelType"])
    file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

    file = file.head(1000) # Limits dataset size

    X = file.drop(columns = ['price'])
    Y = file.price

#     print(file)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)
    
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    print(list(fuelTypeEncoder.classes_))

    return  X_train, X_test, Y_train, Y_test

In [63]:
X_train, X_test, Y_train, Y_test = dataset("UKUsedCarDataSet/audi.csv") # Use Audi dataset as default for KNN analysis

['Diesel', 'Hybrid', 'Petrol']


In [64]:
def rmse(test, pred):
    print("RMSE: ")
    MSE = np.square(np.subtract(test, pred)).mean()
    return sqrt(MSE)

In [65]:
SVM = SVMRegression()
SVM.fit( X_train, Y_train )
Y_pred = SVM.predict( X_test )

In [68]:
def evaluation(train, test, yTrain, yTest):    
    y_pred =  SVM.predict(X_test)
    error = rmse(yTest, y_pred) 
    print('The RMSE value is:', error)
    for i in range(len(y_pred)):
        print("\nOriginal value:", yTest.iloc[i], "vs the predicted value:", y_pred[i])
        print("The difference is:", yTest.iloc[i] - y_pred[i])
evaluation(X_train, X_test, Y_train, Y_test)

RMSE: 
The RMSE value is: 41432.4123298029

Original value: 18498 vs the predicted value: -15213.45362901096
The difference is: 33711.45362901096

Original value: 22995 vs the predicted value: -16812.60810046879
The difference is: 39807.60810046879

Original value: 12495 vs the predicted value: -12513.519925095863
The difference is: 25008.51992509586

Original value: 14500 vs the predicted value: -16661.421392841225
The difference is: 31161.421392841225

Original value: 24995 vs the predicted value: -18279.92401780009
The difference is: 43274.92401780009

Original value: 10200 vs the predicted value: -14821.72780543368
The difference is: 25021.72780543368

Original value: 21995 vs the predicted value: -18141.727625701657
The difference is: 40136.72762570166

Original value: 16800 vs the predicted value: -12587.097969313607
The difference is: 29387.097969313607

Original value: 12798 vs the predicted value: -17247.852313164018
The difference is: 30045.852313164018

Original value: 20498